# Imported Libraries

In [57]:
import numpy as np
from math import gcd, exp, pi
import cmath
import matplotlib.pyplot as plt
from scipy.stats import unitary_group
from sympy.utilities.iterables import partitions
from sympy import symbols, expand, factorial
from collections import defaultdict

# Random Matrix Group Generators

## Random Unitary Matrices

There are two ways we can create a random unitary matrix:
1. The first method is to import the `unitary_group` function from the SciPy library. The documentation can be found [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.unitary_group.html)
2. The second method is to use either the Gram-Schmidt process or the QR decomposition. Essentially, if you create a random complex field matrix, then performing the QR decomposition would result in the unitary matrix $Q$ and the upper triangular matrix $R$ such that $A = QR$

### Method 1: `unitary_group`

In [6]:
unitary_mat = unitary_group.rvs(3)
unitary_mat

array([[-0.26713952-0.40867753j,  0.01151373+0.48350697j,
        -0.15373882-0.70998026j],
       [-0.80725296+0.11684437j, -0.50646481-0.21145205j,
         0.177096  +0.04591836j],
       [-0.26658154+0.15888202j,  0.68172689-0.0108774j ,
         0.64990556-0.12823292j]])

### Method 2: QR Decomposition

In [8]:
dim = 3                                                           # Define the dimension of the complex matrix
A = np.random.randn(dim, dim) + 1j * np.random.randn(dim, dim)    # Create a square matrix where each element is in the Complex Field
Q, R = np.linalg.qr(A)                                            # Find the QR decomposition of matrix A

# Step 3: Check unitarity
print("Q*Q^† ≈ I:\n", np.round(Q @ Q.conj().T, decimals=3))
print("Unitary matrix Q:\n", Q)

Q*Q^† ≈ I:
 [[ 1.-0.j -0.+0.j -0.-0.j]
 [-0.-0.j  1.+0.j  0.+0.j]
 [-0.+0.j  0.+0.j  1.-0.j]]
Unitary matrix Q:
 [[-0.89679643+0.05357511j  0.03837435-0.28535041j  0.33138398-0.01315611j]
 [ 0.01903943+0.1119897j  -0.54227129+0.22665278j  0.32052708+0.73411754j]
 [-0.19363566-0.37747439j -0.70607633+0.27042399j -0.16701116-0.46952388j]]


How Run Gram-Schnidt for each column

### $\sigma$-moment Generating Function

In [10]:
def random_unitary(n):
    """Generate a random unitary matrix using QR decomposition."""
    Z = np.random.randn(n, n) + 1j * np.random.randn(n, n)
    Q, R = np.linalg.qr(Z)
    d = np.diagonal(R)
    ph = d / np.abs(d)
    Q *= ph
    return Q

def sigma_mgf(U, t, k_max=20):
    """
    Compute log Σ(t) = sum_{k=1}^k_max t^k / k * Tr(U^k)
    This is the moment generating function for power sums of eigenvalues.
    """
    n = U.shape[0]
    s = 0.0
    U_power = np.eye(n, dtype=complex)
    for k in range(1, k_max + 1):
        U_power = U_power @ U
        trace = np.trace(U_power)
        s += (t**k / k) * trace
    return s

# Example usage
n = 4
U = random_unitary(n)
t = 0.5
log_sigma = sigma_mgf(U, t)
sigma = np.exp(log_sigma)

print(f"Log Sigma MGF at t={t}: {log_sigma}")
print(f"Sigma MGF at t={t}: {sigma}")


Log Sigma MGF at t=0.5: (0.17264694833278596+0.4236095552300916j)
Sigma MGF at t=0.5: (1.0834010466967812+0.4885152319013758j)


## Random Sympletic Matrices

In [12]:
def random_symplectic_matrix(dim):
    A = np.random.randn(dim, dim)
    B = np.random.randn(dim, dim)
    C = -B.T
    D = A.T
    M = np.block([[A, B], [C, D]])
    return M

In [13]:
n = 2  # for Sp(4, R)
M = random_symplectic_matrix(n)

print("Symplectic matrix M:")
print(np.round(M, 3))

Symplectic matrix M:
[[-2.324  0.632 -0.143 -1.792]
 [ 0.078  1.378 -0.293  0.326]
 [ 0.143  0.293 -2.324  0.078]
 [ 1.792 -0.326  0.632  1.378]]


# $\sigma$-moment generating function for each matrix groups

## Group 1: Cyclic Groups ($C_n$)

### Matrix Reperesentation

In [25]:
import numpy as np

class CyclicGroupMatrixRep:
    def __init__(self, n):
        # Check if n is a positive integer; If not, throw an error since it's not mathematically accurate.
        if not isinstance(n, int) or n <= 0:
            raise ValueError("n must be a positive integer")
        
        self.n = n
        self.generator = self._create_generator()
        self.elements = self._generate_elements()
    
    def _create_generator(self):
        """Create the generator matrix (cyclic permutation matrix)"""
        matrix = np.zeros((self.n, self.n), dtype=int)
        for i in range(self.n - 1):
            matrix[i + 1, i] = 1
        matrix[0, self.n - 1] = 1
        return matrix
    
    def _generate_elements(self):
        """Generate all group elements by powers of the generator"""
        elements = [np.eye(self.n, dtype=int)]  # Start with identity
        
        current = self.generator.copy()
        for _ in range(self.n - 1):
            elements.append(current)
            current = current @ self.generator  # Matrix multiplication
        
        return elements
    
    def get_element(self, k):
        """Get the k-th group element (0 ≤ k < n)"""
        if not 0 <= k < self.n:
            raise ValueError("k must be between 0 and n-1")
        return self.elements[k]
    
    def display_all_elements(self):
        """Display all group elements"""
        print(f"\nAll elements of C_{self.n}:")
        for i, element in enumerate(self.elements):
            print(f"Element {i}:")
            print(element)
            print()

In [27]:
# Example usage
if __name__ == "__main__":
    # Create C₄ matrix representation
    c4 = CyclicGroupMatrixRep(3)
    
    print("Generator matrix:")
    print(c4.generator)
    
    # Display all elements
    c4.display_all_elements()

Generator matrix:
[[0 0 1]
 [1 0 0]
 [0 1 0]]

All elements of C_3:
Element 0:
[[1 0 0]
 [0 1 0]
 [0 0 1]]

Element 1:
[[0 0 1]
 [1 0 0]
 [0 1 0]]

Element 2:
[[0 1 0]
 [0 0 1]
 [1 0 0]]



### $\sigma$-moment generating function

In [131]:
class CyclicGroup:
    # Initialize the cyclic group Z/nZ representation
    def __init__(self, n):
        # Confirm the group order to be a positive integer
        # If not, throw an error since it's mathematically inaccurate
        if not isinstance(n, int) or n <= 0:
            raise ValueError("n must be a positive integer")
        # Initialize the group's order
        self.n = n

    def chi_k(self, k, m):
        """
        Compute the function chi_k(m) = e^(2i*pi*m*k/n)
        
        Parameters:
        - k: ???
        - m: ???
        """
        # Return the computation function for chi_k(m) based on the proof's formula
        return cmath.exp(2j * pi * k * m / self.n)
    
    def expectation_func(self, k, tau):
        """
        Compute E[X_{p_tau}]. In this case, the proof states that E[X_{p_tau}] can be 
        interpreted piecewise based on the divisibility between n and k * |tau|, where 
        |tau| denotes the sum of all the entries of tau.

        Parameters:
        - tau: a list of [alpha_1, alpha_2, alpha_3,...,alpha_m] 
        """
        # |tau| denotes the sum of all the entries of tau
        tau_sum = np.sum(tau)
        # If k * tau_sum is divisible by n, return 1
        if (k * tau_sum) % self.n == 0:
            return 1
        # If not, return 0.
        else:
            return 0
    
    def sigma_mgf(self, k, max_degree=None):
        """
        Compute the sigma-moment generating function E[exp_sigma(Xh_1)] where
        E[exp_sigma(Xh_1)] = Sigma_τ ((delta_{n,k,tau}/z_tau)*p_tau).
        
        Parameters:
        - k: ???
        - max_degree: maximum degree of partitions to consider
        """
        if max_degree is None:
            max_degree = self.n
        
        result = defaultdict(float)
        
        for tau in self.partitions(max_degree):
            # We interpret the expression for E[X_{p_tau}] as delta_{n,k,tau},
            # which means that delta = result of expectation_func.
            delta = self.expectation_func(k, tau)
            
            # Calculate z_tau and p_tau
            z_tau = self.z_tau(tau)
            p_tau = self.p_tau(k, tau)

            # Calculate the sigma-moment function
            result[tau] = (delta * p_tau) / z_tau
        
        return result

    def z_tau(self, tau):
        """
        According to Prof. Sean's paper (page 4), z_tau is computed similarly to p_tau,
        h_tau, and e_tau. The paper states that p_tau = prod(p_{tau_j}), h_tau = prod(h_{tau_j}),
        and e_tau = prod(e_{tau_j}). Hence, z_tau = prod(z_{tau_j}).

        Parameters:
        - tau: a list of alpha values [alpha_1, alpha_2, alpha_3,...,alpha_m] 
        """
        # Create a dictionary that keeps track of how many alpha values are in tau.
        multiplicity = defaultdict(int)
        for alpha in tau:
            multiplicity[alpha] += 1

        # Compute the multiplicity of z_tau
        z = 1
        for i, m in multiplicity.items():
            z *= (i**m) * factorial(m)
        return z

    def p_tau(self, k, tau):
        """
        Compute p_tau = prod(p_{tau_j}), where p_{tau_j} = prod_{m} chi_k(m)^{tau_j}
        
        Parameters:
        - tau: a list of alpha values [alpha_1, alpha_2, alpha_3,...,alpha_m] 
        """
        # Compute the multiplicity of p_tau
        p = 1
        for alpha in tau:
            p_tau_j = sum(self.chi_k(k, m)**alpha for m in range(self.n))
            p *= p_tau_j
        return p

    def partitions(self, max_degree):
        # Generate integer partitions up to max_degree
        for size in range(max_degree + 1):
            for p in partitions(size):
                partition = []
                for part, mult in sorted(p.items(), reverse=True):
                    partition.extend([part] * mult)
                yield tuple(partition)
        yield tuple() 

    
    def lambda_distribution(self, k, f_expansion):
        """
        Compute the Lambda-distribution mu_X(f) = sum_{lambda: n | k*|lambda|} α_lambda.
        
        Parameters:
        - k: ???
        - f_expansion: dict {partition: coefficient} representing f in power sum basis
        """
        d = self.n // gcd(self.n, k)
        return sum(alpha for Lambda, alpha in f_expansion.items() 
                 if sum(Lambda) % d == 0)

In [133]:
# Example usage
if __name__ == "__main__":
    # Initialize with n=6
    n = 6
    cgr = CyclicGroup(n)
    
    # Example 1: Compute character values
    k = 2
    print(f"Character values χ_{k} for Z/{n}Z:")
    for m in range(n):
        char_value = cgr.chi_k(k, m)
        # Format complex number nicely
        real_part = char_value.real
        imag_part = char_value.imag
        if abs(real_part) < 1e-10:
            real_part = 0
        if abs(imag_part) < 1e-10:
            imag_part = 0
        print(f"χ_{k}({m}) = {real_part:.3f} + {imag_part:.3f}i")
    
    # Example 2: Compute expectation for various τ
    test_cases = [
    ((1,), "τ = (1)"),
    ((2,), "τ = (2)"),
    ((1, 1), "τ = (1,1)"),
    ((3,), "τ = (3)"),
    ((2, 1), "τ = (2,1)"),
    ((1, 1, 1), "τ = (1,1,1)")
    ]
    
    print("\nExpectation values 𝔼[X_pτ] for k=2:")
    for tau, desc in test_cases:
        expectation = cgr.expectation_func(k, tau)
        print(f"{desc}: {expectation}")
    
    # Example 3: Compute moment generating function terms
    print("\nMoment generating function terms (up to degree 3):")
    mgf = cgr.sigma_mgf(k, max_degree=3)
    for tau, coeff in sorted(mgf.items()):
        print(f"p{tau}: {coeff:.3f}")
    
    # Example 4: Compute Λ-distribution
    # Let's create a test symmetric function f = p₁ + 2p₂ + p_(1,1)
    f_expansion = {
        (1,): 1,
        (2,): 2,
        (1, 1): 1
    }
    
    print("\nΛ-distribution for f = p₁ + 2p₂ + p_(1,1):")
    for k_test in [0, 1, 2, 3]:
        result = cgr.lambda_distribution(k_test, f_expansion)
        print(f"k={k_test}: μ_X(f) = {result}")

Character values χ_2 for Z/6Z:
χ_2(0) = 1.000 + 0.000i
χ_2(1) = -0.500 + 0.866i
χ_2(2) = -0.500 + -0.866i
χ_2(3) = 1.000 + 0.000i
χ_2(4) = -0.500 + 0.866i
χ_2(5) = -0.500 + -0.866i

Expectation values 𝔼[X_pτ] for k=2:
τ = (1): 0
τ = (2): 0
τ = (1,1): 0
τ = (3): 1
τ = (2,1): 1
τ = (1,1,1): 1

Moment generating function terms (up to degree 3):
p(): 1.000
p(1,): 0.000
p(1, 1): 0.000
p(1, 1, 1): 0.000
p(2,): 0.000
p(2, 1): 0.000
p(3,): 2.000

Λ-distribution for f = p₁ + 2p₂ + p_(1,1):
k=0: μ_X(f) = 4
k=1: μ_X(f) = 0
k=2: μ_X(f) = 0
k=3: μ_X(f) = 3


## Group 2: Dihedral Groups ($D_n$)

### Matrix Reperesentation and Calculated Expectation

In [19]:
# Create a class of Dihedral Group that computes the sigma-moment generating function. 
class DihedralGroup:
    def __init__(self, n, k):
        """ 
        Initialize the dihedral group representation
    
        Parameters:
        - n: the order of the dihedral group D_n
        - k: ???
        """
        self.n = n
        self.k = k
        self.z = exp(2j * pi * k / n)

    def rho_k(self, rotation, l):
        """
        Returns a 2x2 matrix representation based on rho_k(n)
        
        Parameters:
        - rotation: return the respective matrix based on two rotation options:
                  'r' is to rotate l times (R_n(l))
                  'f' is to both rotate l times and 1 flip (F_n(l))
        - l: the number of rotations performed
        """
        sigma = (2 * pi * self.k * l) / self.n
        if rotation == 'r':  
            return np.array([
                [exp(1j * sigma), 0],
                [0, exp(-1j * sigma)]
            ], dtype=complex)
        elif rotation == 'f':
            return np.array([
                [0, exp(1j * sigma)],
                [exp(-1j * sigma), 0]
            ], dtype=complex)
        else:
            raise ValueError("Element type must be 'r' or 'f'")

    def eigenvalues(self, rotation, l):
        """
        Compute the eigenvalues for each type of matrix rotation
        
        Parameters:
        - rotation: return the respective matrix based on two rotation options:
                  'r' is to rotate l times (R_n(l))
                  'f' is to both rotate l times and 1 flip (F_n(l))
        - l: int - number of rotations
        """
        theta = (2 * pi * self.k * l) / self.n
        if rotation == 'r':
            return [exp(1j * theta), exp(-1j * theta)]
        elif rotation == 'f':
            return [1, -1]
        else:
            raise ValueError("Element type must be 'r' or 'f'")

    def sigma_mgf(self, tau, l, rotation='r'):
        """
        Compute the sigma-moment generating function of both R_n(l) and F_n(l). 
        We set R_n(l) as the default computation since F_n(l) would always return 0.
        
        Parameters:
        - rotation: return the respective matrix based on two rotation options:
          'r' is to rotate l times (R_n(l))
          'f' is to both rotate l times and 1 flip (F_n(l))
        - tau: a list of [alpha_1, alpha_2, alpha_3,...,alpha_m]
        - l: int - number of rotations
        """
        if rotation == 'r':
            eigen_val = self.eigenvalues(rotation, l)
            p_tau_result = 1
            for alpha in tau:
                term = (eigen_val[0]**alpha) + (eigen_val[1]**alpha)
                p_tau_result *= term
            return p_tau_result
        elif rotation == 'f':
            return 0
        else:
            raise ValueError("Element type must be 'r' or 'f'")

    def expectation_func(self, tau):
        """
        Compute E[X_{p_tau}]

        Parameters:
        - tau: a list of [alpha_1, alpha_2, alpha_3,...,alpha_m] 
        """
        res = 0
        for l in range(self.n):
            res += self.sigma_mgf(tau, l)
        return res / (2 * self.n)

In [20]:
# Create D₄ group with representation k=1
d4 = DihedralGroup(n=4, k=1)
tau = [1, 2]  # Our parameters

print("=== Matrix Representations in D₄ ===")
print("Rotation r^1:")
print(d4.rho_k('r', 1))
print("\nFlip f (r^0 f):")
print(d4.rho_k('f', 0))

print("\n=== Eigenvalues ===")
print("Eigenvalues of r^1:", [f"{x:.3f}" for x in d4.eigenvalues('r', 1)])
print("Eigenvalues of f:", d4.eigenvalues('f', 0))

print("\n=== σ-Moment Generating Function ===")
print(f"For τ = {tau}:")
for l in range(4):
    print(f"r^{l}: {d4.sigma_mgf(tau, l):.3f}")
print(f"f: {d4.sigma_mgf(tau, 0, 'f')}")  # Should be 0

print("\n=== Expectation Value ===")
expectation = d4.expectation_func(tau)
print(f"𝔼[X_{tau}] = {expectation:.3f}")

=== Matrix Representations in D₄ ===
Rotation r^1:
[[6.123234e-17+1.j 0.000000e+00+0.j]
 [0.000000e+00+0.j 6.123234e-17-1.j]]

Flip f (r^0 f):
[[0.+0.j 1.+0.j]
 [1.-0.j 0.+0.j]]

=== Eigenvalues ===
Eigenvalues of r^1: ['0.000+1.000j', '0.000-1.000j']
Eigenvalues of f: [1, -1]

=== σ-Moment Generating Function ===
For τ = [1, 2]:
r^0: 4.000+0.000j
r^1: -0.000+0.000j
r^2: -4.000+0.000j
r^3: 0.000-0.000j
f: 0

=== Expectation Value ===
𝔼[X_[1, 2]] = 0.000+0.000j


## Group 3 & 4: Symmetric Group $S_n$ and Alternating Group $A_n$